In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
!pip install git+https://github.com/qubvel/classification_models.git

  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-5r2x8th3
  Running command git clone -q https://github.com/qubvel/classification_models.git /tmp/pip-req-build-5r2x8th3
  Running command git submodule update --init --recursive -q
  Created wheel for image-classifiers: filename=image_classifiers-1.0.0-cp36-none-any.whl size=20032 sha256=5a0c5df7f797827343394cb8f6449b3a8b16a1d504514b0eef7a12c1bd0ddd69
  Stored in directory: /tmp/pip-ephem-wheel-cache-ekh19vi7/wheels/de/2b/fd/29a6d33edb8c28bc7d94e95ea1d39c9a218ac500a3cfb1b197
Successfully built image-classifiers


In [5]:
!pip install efficientnet

In [7]:
!pip install iterative-stratification

In [1]:
import pandas as pd
import numpy as np
import os
import gc
from classification_models.keras import Classifiers
import efficientnet
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Dense, Dropout, BatchNormalization, concatenate
from keras.models import Model
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import keras
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import albumentations
from tqdm import tqdm
import warnings
from albumentations import Resize, CenterCrop, Rotate, HorizontalFlip, VerticalFlip, OneOf, Compose, ShiftScaleRotate

Using TensorFlow backend.


In [0]:
BASE_PATH = 'gdrive/My Drive/kaggle/'
train_df = pd.read_csv(os.path.join(BASE_PATH,'train.csv'))
test_df = pd.read_csv(os.path.join(BASE_PATH,'test.csv'))
class_map_df = pd.read_csv(os.path.join(BASE_PATH,'class_map.csv'))
sample_sub_df = pd.read_csv(os.path.join(BASE_PATH,'sample_submission.csv'))

In [3]:
tgt_cols = ['grapheme_root','vowel_diacritic','consonant_diacritic']
desc_df = train_df[tgt_cols].astype('str').describe()
desc_df

,grapheme_root,vowel_diacritic,consonant_diacritic
count,200840,200840,200840
unique,168,11,7
top,72,0,0
freq,5736,41508,125278


In [0]:
types = desc_df.loc['unique',:]

In [5]:
train_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [0]:
# TRAIN = [BASE_PATH + 'train_image_data_0.parquet',
#          BASE_PATH + 'train_image_data_1.parquet',
#          BASE_PATH + 'train_image_data_2.parquet',
#          BASE_PATH + 'train_image_data_3.parquet']

# img_df1 = pd.read_parquet(TRAIN[0])
# img_df1 = img_df1.drop(['image_id'], axis = 1)
# img_df1 = img_df1.values
# np.save(BASE_PATH + 'train_images1.npy', img_df1)
# del img_df1
# gc.collect()
# img_df2 = pd.read_parquet(TRAIN[1])
# img_df2 = img_df2.drop(['image_id'], axis = 1)
# img_df2 = img_df2.values
# np.save(BASE_PATH + 'train_images2.npy', img_df2)
# del img_df2
# gc.collect()
# img_df3 = pd.read_parquet(TRAIN[2])
# img_df3 = img_df3.drop(['image_id'], axis = 1)
# img_df3 = img_df3.values
# np.save(BASE_PATH + 'train_images3.npy', img_df3)
# del img_df3
# gc.collect()
# img_df4 = pd.read_parquet(TRAIN[3])
# img_df4 = img_df4.drop(['image_id'], axis = 1)
# img_df4 = img_df4.values
# np.save(BASE_PATH + 'train_images4.npy', img_df4)
# del img_df4
# gc.collect()

In [0]:
root = train_df["grapheme_root"].values
vowel = train_df["vowel_diacritic"].values
consonent = train_df["consonant_diacritic"].values

In [9]:
data = np.load(BASE_PATH + "train_images1.npy")
#AUGMENTATIONS
ss = ShiftScaleRotate(rotate_limit=12)
cc = CenterCrop(p = 1.0, height=120, width=210)
rs = Resize(height=100, width=100, p=1.0)
hf = HorizontalFlip()
vf = VerticalFlip()
augs = Compose([ss, cc, rs, hf, vf], p = 1.0)
label_index = 0 #+ 50210 + 50210 + 50210
for k in tqdm(range(0, data.shape[0]), total=data.shape[0]):
    x = data[k, :]
    x = np.reshape(x, newshape=(137, 236))
    n = augs(image=x)["image"]
    name = "train_images_100_100/" + str(label_index+k) + "_" + str(root[label_index+k])+"_"+str(vowel[label_index+k])+"_"+str(consonent[label_index+k])+".npy"
    np.save(file=name, arr=n)

100%|██████████| 50210/50210 [00:40<00:00, 1247.07it/s]


In [10]:
del data
gc.collect()

0

In [11]:
data = np.load(BASE_PATH + "train_images2.npy")
#AUGMENTATIONS
ss = ShiftScaleRotate(rotate_limit=12)
cc = CenterCrop(p = 1.0, height=120, width=210)
rs = Resize(height=100, width=100, p=1.0)
hf = HorizontalFlip()
vf = VerticalFlip()
augs = Compose([ss, cc, rs, hf, vf], p = 1.0)
label_index = 0 + 50210 #+ 50210 + 50210
for k in tqdm(range(0, data.shape[0]), total=data.shape[0]):
    x = data[k, :]
    x = np.reshape(x, newshape=(137, 236))
    n = augs(image=x)["image"]
    name = "train_images_100_100/" + str(label_index+k) + "_" + str(root[label_index+k])+"_"+str(vowel[label_index+k])+"_"+str(consonent[label_index+k])+".npy"
    np.save(file=name, arr=n)
del data
gc.collect()

100%|██████████| 50210/50210 [00:55<00:00, 902.66it/s]


0

In [12]:
data = np.load(BASE_PATH + "train_images3.npy")
#AUGMENTATIONS
ss = ShiftScaleRotate(rotate_limit=12)
cc = CenterCrop(p = 1.0, height=120, width=210)
rs = Resize(height=100, width=100, p=1.0)
hf = HorizontalFlip()
vf = VerticalFlip()
augs = Compose([ss, cc, rs, hf, vf], p = 1.0)
label_index = 0 + 50210 + 50210 #+ 50210
for k in tqdm(range(0, data.shape[0]), total=data.shape[0]):
    x = data[k, :]
    x = np.reshape(x, newshape=(137, 236))
    n = augs(image=x)["image"]
    name = "train_images_100_100/" + str(label_index+k) + "_" + str(root[label_index+k])+"_"+str(vowel[label_index+k])+"_"+str(consonent[label_index+k])+".npy"
    np.save(file=name, arr=n)
del data
gc.collect()

100%|██████████| 50210/50210 [00:39<00:00, 1269.72it/s]


25

In [13]:
data = np.load(BASE_PATH + "train_images4.npy")
#AUGMENTATIONS
ss = ShiftScaleRotate(rotate_limit=12)
cc = CenterCrop(p = 1.0, height=120, width=210)
rs = Resize(height=100, width=100, p=1.0)
hf = HorizontalFlip()
vf = VerticalFlip()
augs = Compose([ss, cc, rs, hf, vf], p = 1.0)
label_index = 0 + 50210 + 50210 + 50210
for k in tqdm(range(0, data.shape[0]), total=data.shape[0]):
    x = data[k, :]
    x = np.reshape(x, newshape=(137, 236))
    n = augs(image=x)["image"]
    name = "train_images_100_100/" + str(label_index+k) + "_" + str(root[label_index+k])+"_"+str(vowel[label_index+k])+"_"+str(consonent[label_index+k])+".npy"
    np.save(file=name, arr=n)
del data
gc.collect()

100%|██████████| 50210/50210 [00:55<00:00, 910.27it/s]


25

In [14]:
FOLDER_TRAIN = "train_images_100_100/"
files = os.listdir(FOLDER_TRAIN)
data = pd.DataFrame()
data["file_names"] = files
data["file_names"] = FOLDER_TRAIN + data["file_names"]
data.head()

,file_names
0,train_images_100_100/134794_18_3_0.npy
1,train_images_100_100/95033_22_1_0.npy
2,train_images_100_100/131707_65_3_2.npy
3,train_images_100_100/196501_107_4_4.npy
4,train_images_100_100/155010_38_5_0.npy


In [15]:
x = data["file_names"].str.split(".", n = 1, expand = True)
x = x[x.columns[0]]
x = x.str.split("_", n = 7, expand = True)
data["root"] = x[x.columns[4]]
data["vowel"] = x[x.columns[5]]
data["consonent"] = x[x.columns[6]]
data.head()

,file_names,root,vowel,consonent
0,train_images_100_100/134794_18_3_0.npy,18,3,0
1,train_images_100_100/95033_22_1_0.npy,22,1,0
2,train_images_100_100/131707_65_3_2.npy,65,3,2
3,train_images_100_100/196501_107_4_4.npy,107,4,4
4,train_images_100_100/155010_38_5_0.npy,38,5,0


In [0]:
X = data["file_names"].values
y = data[["root", "vowel", "consonent"]].values

In [17]:
mskf = MultilabelStratifiedKFold(n_splits=5, random_state=0)
train_indexes = []
test_indexes = []

for train_index, test_index in mskf.split(X, y):
    train_indexes.append(train_index)
    test_indexes.append(test_index)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [18]:
del data, x, files
gc.collect()

181

In [0]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(100, 100), n_channels=1, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.list_IDs_IND = range(0, len(list_IDs))
        
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs_IND[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))

        y1 = np.empty((self.batch_size), dtype=int)
        y2 = np.empty((self.batch_size), dtype=int)
        y3 = np.empty((self.batch_size), dtype=int)

        # Generate data
        
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            #print (ID)
            X[i, :, :, 0] = np.load(self.list_IDs[ID]) / 255.0

            # Store class
            temp_y = self.labels[ID]
            #print(temp_y)
            y1[i] = temp_y[0]
            y2[i] = temp_y[1]
            y3[i] = temp_y[2]
        y1 = keras.utils.to_categorical(y1, num_classes=168)
        y2 = keras.utils.to_categorical(y2, num_classes=11)
        y3 = keras.utils.to_categorical(y3, num_classes=7)
        
        return X, [y1, y2, y3]

In [0]:
HEIGHT = 100
WIDTH = 100
CHANNELS = 1

In [0]:
def create_model(BACKBONE, HEIGHT, WIDTH, CHANNELS, WEIGHTS):
    
    arch, _ = Classifiers.get(BACKBONE)
    model = arch((HEIGHT, WIDTH, CHANNELS), weights=WEIGHTS, include_top=False)
    x_Avg = GlobalAveragePooling2D()(model.output)
    x_Max = GlobalMaxPooling2D()(model.output)
    concat = concatenate([x_Avg, x_Max])
    concat = BatchNormalization()(concat)
    concat = Dropout(0.5)(concat)
    #ROOT
    x = Dense(512)(concat)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    root = Dense(168, activation='sigmoid', name='root')(x)

    #VOWEL
    x = Dense(512)(concat)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    vowel = Dense(11, activation='sigmoid', name='vowel')(x)

    #CONSONENT
    x = Dense(512)(concat)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    consonent = Dense(7, activation='sigmoid', name='consonent')(x)

    model = Model(inputs = [model.input], outputs=[root, vowel, consonent])
    return model

In [22]:
#FIRST FOLD
FOLD_NUM = 0
X_train, X_test = X[train_indexes[FOLD_NUM]], X[test_indexes[FOLD_NUM]]
y_train, y_test = y[train_indexes[FOLD_NUM]], y[test_indexes[FOLD_NUM]]
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((160672,), (40168,), (160672, 3), (40168, 3))

In [0]:
train_generator = DataGenerator(list_IDs=X_train, labels=y_train, batch_size=32)
valid_generator = DataGenerator(list_IDs=X_test, labels=y_test, batch_size=32)

In [0]:
model = create_model(BACKBONE="resnet18", HEIGHT=HEIGHT, WIDTH=WIDTH, CHANNELS=CHANNELS, WEIGHTS=None)

In [0]:
checkpointer = ModelCheckpoint(filepath = 'model_best_root_loss_fold_50' + str(FOLD_NUM) + '.h5',
                               verbose=1,
                               save_best_only=True, save_weights_only = True)

reduce_lr = ReduceLROnPlateau(monitor='val_root_loss', factor=0.8,
                              patience=10, min_lr=0.000001, verbose=1, cooldown=1)

In [0]:
model.compile(optimizer=keras.optimizers.Adam(), 
              loss={
                  'root' :'binary_crossentropy',
                   'vowel' :'binary_crossentropy',
                   'consonent' :'binary_crossentropy',
                  }, 
                loss_weights = {
                    'root' : 0.5,
                   'vowel' : 0.35,
                   'consonent' : 0.15,
                },
              metrics=['accuracy'])

In [0]:
model.fit_generator(generator=train_generator, epochs=50, validation_data=valid_generator)



Epoch 1/50
5021/5021 [==============================] - 390s 78ms/step - loss: 0.1306 - root_loss: 0.0522 - vowel_loss: 0.1896 - consonent_loss: 0.2545 - root_acc: 0.9878 - vowel_acc: 0.9295 - consonent_acc: 0.8982 - val_loss: 0.1113 - val_root_loss: 0.0399 - val_vowel_loss: 0.1767 - val_consonent_loss: 0.1968 - val_root_acc: 0.9940 - val_vowel_acc: 0.9410 - val_consonent_acc: 0.9217
Epoch 2/50
3668/5021 [====================>.........] - ETA: 1:32 - loss: 0.0675 - root_loss: 0.0315 - vowel_loss: 0.0871 - consonent_loss: 0.1416 - root_acc: 0.9940 - vowel_acc: 0.9675 - consonent_acc: 0.9449